In [10]:
import numpy as np
import pandas as pd
import os
from os import listdir
months = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205']
# months = ['202205']

if not os.path.isdir('POP_cooked_data'):
    os.mkdir('POP_cooked_data')

In [11]:
# pop_raw = pd.read_csv(f"POP_raw_data/雙北里人口資料/202205.csv") 

stn_vil = pd.read_csv('POP_raw_data/站與里/站與里_全.csv') # info of possible serving area (surrounding villages) of a station
firm_count = pd.read_csv('FIRM_raw_data/tp_firm_count.csv') # info of company counts of a certain village
school_count = pd.read_csv('SCH_raw_data/school_count.csv') # info of school counts of a certain village

# pd.unique(pop_raw[['區','里']].values.ravel('K')).shape # show unique values of administrative area column and village column

In [12]:
for m in months:
    pop_raw = pd.read_csv(f"POP_raw_data/雙北里人口資料/{m}.csv") # population data of each village in Tapei City and New Taipei City
    cooked = pd.merge(stn_vil,pop_raw, how='left', on=['區','里']) # merge 'stn_vil' and 'pop_raw' on administrative area name and village name
    cooked = pd.merge(cooked,firm_count, how='left', on=['區','里']) # merge 'firm_count' on administrative area name and village name
    cooked = pd.merge(cooked,school_count, how='left', on=['區','里']) # merge 'school_count' on administrative area name and village name
    
    cooked['公司數'] = cooked['公司數'].fillna(0) 
    cooked['學校數'] = cooked['學校數'].fillna(0)
    
    cooked = cooked.groupby(['站名']).sum()
    index = pd.DataFrame([*cooked.index]) # convert index into dataframe
    value = pd.DataFrame(cooked.values) # convert value into dataframe
    cooked = pd.concat([index, value], axis=1) # concatenate index and value into a dataframe
    cooked.columns = ['站名','人口數','男','女','公司數','學校數']
    
    cooked.to_csv(f"POP_cooked_data/cooked_{m}.csv", index=False)